## EDA on WB pop data: basic manipulations and outliers 

Each section links to some of the questions that were tasks in class.

### Importing packages & data

Rremember: relative paths are better for collaboration.


In [41]:
# import libraries
import pandas as pd
import os
import matplotlib.pyplot as plt 
import seaborn as sns  # remember to install this in your environment first

In [42]:
# import data - get current working directory and its parent
cwd = os.getcwd()
parent_path = os.path.dirname(cwd)

df_og = pd.read_csv(parent_path + '/data/WB_pop_clean.csv')

### Distribution and outliers

- Are there any outliers?
    - Show them in a box plot.
    - What could you do with them?
- Can you plot a histogram and/ or kernel density for the values in 2023?

In [43]:
# it's good practice to make a copy of the data set before you start manipulating it
# that way you also don't have to import the data from scratch each time you made an error (e.g. removing a column you end up wanting to keep)

df = df_og.copy()

In [ ]:
df.columns

In [ ]:
years_choice = ['1990', '2020', '2024']
df.boxplot(column = years_choice, grid = False) # this is native to pandas which uses matplotlib without us having to call it
plt.show()

In [ ]:
# we know the "world" row is the outlier, which is not a country, so let's remove it
# remember: we can just re-run the cell that copies the og data to undo this change)

# option 1: only keep other rows
df = df[df['Country Name'] != 'World']

# option 2: drop specific row
#df = df.drop(df.loc[df['Country Name'] == 'World'])

# option 3: see the option with ~ below (just search for the tilde with ctr+f)

# check that there is one row less:
print(df.shape)


In [ ]:
# plot again, this time with seaborn, and you can watch India overtake China

sns.set_style("whitegrid") 

columns_to_plot = years_choice
  
sns.boxplot(data = df[columns_to_plot]) 
plt.show()

In [ ]:
# play with the bins variable of the histogram
df.hist(column=['2023'], bins=20)
plt.show()

In [ ]:
# kernel densities also give an idea of the overall distribution

df['2023'].plot.density()
plt.show()

### Population growth over time


- Which county has seen the greatest population growth from the start to the end of the timeline?
    - In absolute terms?
    - In relative terms?
    - What about the greatest decrease?

In [ ]:
# create a new column that shows the difference between two years of your choice- note that the absolute value would not be helpful

year1 = '1990'
year2 = '2024'

df[f'gwth_{year1}_{year2}_abs'] = df[year2] - df[year1] 
df[f'gwth_{year1}_{year2}_rel'] = round(df[f'gwth_{year1}_{year2}_abs'] / df[year1], 2)
df.head(2)

In [ ]:
# find the highest growth values - you can of course to the equivalent for the smallest (i.e. most negative) values
# bonus: can you find the countries with the samlles positive growth?

print(f'Greatest absolute growth from {year1} to {year2}: ')
print('   - Country: ', df.nlargest(1,f'gwth_{year1}_{year2}_abs')['Country Name'])
print('   - Growth: ', df.nlargest(1, f'gwth_{year1}_{year2}_abs')[f'gwth_{year1}_{year2}_abs'])

print('\n') # this prints a line break

print(f'Greatest relative growth from {year1} to {year2}: ')
print('   - Country: ', df.nlargest(1, f'gwth_{year1}_{year2}_rel')['Country Name'])
print('   - Growth: ', df.nlargest(1, f'gwth_{year1}_{year2}_rel')[f'gwth_{year1}_{year2}_rel'])

##### Comparing rows (countries)


Addresses the following questions from class:
- Show just the rows for Spain and your country/ countries of origin
- Can you create a new row that shows the difference in population over time?
- Can you plot this?

In [ ]:
# displaying only selected countries

df.loc[( df['Country Name']== 'Germany') | (df['Country Name']== 'United Kingdom') | (df['Country Name']== 'Spain')]

In [ ]:
# slightly neater and more flexible

country_choices = ['Germany', 'United Kingdom', 'Spain']

df.loc[df['Country Name'].isin(country_choices)].sort_values('2023', ascending=False) # here I am also specifying what to sort by

##### Using transpose to compare columns

In [ ]:
# given the original format of the data, the easiest way to comoare values witin a column is transpose the data
# to work only with the subset we can store it in a new data frame

# country_choices = ['Germany', 'United Kingdom', 'Spain'] # this was already set above

df_subset = df.loc[df['Country Name'].isin(country_choices) ]

print(df_subset.shape)
df_subset

In [ ]:
# this is what tranposing does - note the nice column names thanks to changing the index
df_transposed = df_subset.T
df_transposed

In [ ]:
# I can drop the repeated information to make it neater

to_drop = ['Country Code', 'Series Name', 'Series Code', 'Country Name'] # without this the transpose creates repeated values
df_subset2 = df_subset.drop(columns=to_drop)

df_transposed = pd.DataFrame(df_subset2.T)
df_transposed

In [ ]:
# now finding the differences is easier
df_transposed['diff_deu_esp'] = df_transposed['DEU'] - df_transposed['ESP']
df_transposed['diff_deu_gbr'] = df_transposed['DEU'] - df_transposed['GBR']
df_transposed

#### less elegant

In [ ]:
import numpy as np 

# to take the difference row-wise, I need to select only the columns that contain numbers and are relevant

year_cols = ['1990', '2000', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']# 2023 data: ['2021', '2002', '2003', '2011', '2012', '2013', '2021', '2022', '2023']

df = df.copy() # otherwise you'll get a warning about assinging values that might overwrite existing ones

# we can also make the index (i.e. the row name) more meaningful by inserting the country code
df.set_index(df['Country Code'], inplace=True)

# create rows if they don't exist
if 'diff_deu_gbr' not in df.index:
    df.loc['diff_deu_gbr'] = np.nan # making the default value Nan

if 'diff_deu_esp' not in df.index:
    df.loc['diff_deu_esp'] = 0      # making the default value 0

for col in year_cols:
    df.loc['diff_deu_gbr', col] = df.loc['DEU', col] - df.loc['GBR', col]
    df.loc['diff_deu_esp', col] = df.loc['DEU', col] - df.loc['ESP', col]

df.loc[['DEU', 'GBR', 'ESP'], ['Country Name', 'Series Name'] + year_cols ]


In [ ]:

# plotting from rows
rows_to_plot = ['diff_deu_gbr', 'diff_deu_esp']
numerical_cols =  year_cols #['2001', '2002', '2003', '2011', '2012', '2013', '2021', '2022', '2023']


data_to_plot = df.loc[rows_to_plot, numerical_cols]

# transpose the data to make the columns the x-axis (years) and the rows the data series
data_to_plot = data_to_plot.T

# plot the data
data_to_plot.plot(kind='line', marker='o', figsize=(10, 6))

# specify labels and title
plt.title('How much smaller UK and Spain are compared to Germany')
plt.xlabel('Year')
plt.ylabel('Values')
plt.legend(title='Country/Row')

# show the plot
plt.show()